In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# callbacks

Callbacks implemented in the fastai library

- LRFinder
- OneCycleScheduler
- MixUpCallback
- CSVLogger
- GeneralScheduler
- MixedPrecision
- HookCallback
- RNNTrainer
- TerminateOnNanCallback
- EarlyStoppingCallback
- SaveModelCallback
- ReduceLROnPlateauCallback
- PeakMemMetric
- StopAfterBatches

other callbacks for train and basic_train

- Recorder
- ShowGraph
- BnFreeze
- GradientClipping

In [ ]:
from fastai.vision import *
from fastai.callbacks import *

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)

In [ ]:
data = ImageDataBunch.from_folder(path)

In [ ]:
def simple_learn():
    return Learner(data, simple_cnn((3, 16, 16, 2)), metrics=[accuracy])
learn = simple_learn()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()
learn.recorder.plot_losses()

In [ ]:
lr = 1e-1
learn.fit_one_cycle(3, lr)

In [ ]:
learn.recorder.plot_lr(show_moms=True)

In [ ]:
learn = Learner(data, simple_cnn((3, 16, 16, 2)), metrics=[accuracy]).mixup()
learn.fit_one_cycle(3)

In [ ]:
learn = Learner(data, simple_cnn((3, 16, 16, 2)), metrics=[accuracy], callback_fns=[ShowGraph]).mixup()
learn.fit_one_cycle(10, lr)

In [ ]:
learn = Learner(data, simple_cnn((3, 16, 16, 2)), metrics=[accuracy, error_rate], callback_fns=[CSVLogger])
learn.fit_one_cycle(3, lr)

In [ ]:
learn.csv_logger.read_logged_file()

In [ ]:
def fit_odd_schedule(learn, lr):
    n = len(learn.data.train_dl)
    phases = [TrainingPhase(n).schedule_hp('lr', lr, anneal=annealing_cos),
             TrainingPhase(n).schedule_hp('lr', lr, anneal=annealing_poly(2))]
    sched = GeneralScheduler(learn, phases)
    learn.callbacks.append(sched)
    total_epochs = 10
    learn.fit_one_cycle(total_epochs)

In [ ]:
leanr = Learner(data, simple_cnn((3, 16, 16, 2)), metrics=[accuracy, error_rate])
fit_odd_schedule(learn, 1e-3)

In [ ]:
learn.recorder.plot()
learn.recorder.plot_losses()
learn.recorder.plot_lr()

In [ ]:
learn = Learner(data, simple_cnn((3, 16, 16, 2)), metrics=[accuracy])
learn.fit_one_cycle(3, 1e-4, callbacks=[SaveModelCallback(learn, every='epoch', monitor='accuracy')])

In [ ]:
! ls ~/.fastai/data/mnist_sample/models

In [ ]:
learn = Learner(data, simple_cnn((3, 16, 16, 2)), metrics=accuracy, callback_fns=ActivationStats)
learn.fit(2)

In [ ]:
len(learn.data.train_dl)

In [ ]:
len(learn.activation_stats.modules)

In [ ]:
learn.activation_stats.stats.shape

In [ ]:
# standard deviation

import matplotlib.pyplot as plt
plt.plot(learn.activation_stats.stats[1][-2].numpy())

In [2]:
from fastai.callbacks import *
from fastai.vision import *
import torch

In [4]:
def model_size(m, size):
    x = m(torch.zeros(1, in_channels(m), *size))
    return [o.stored.shape for o in hook_output(m)]